In [ ]:
pip install opencv-python numpy matplotlib pytube3 librosa soundfile

In [ ]:
pip install sounddevice

In [ ]:
pip install pytube moviepy

In [ ]:
pip install pafy pydub


In [ ]:
pip install pyAudioAnalysis


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import sounddevice as sd
from multiprocessing import Process, Queue
import subprocess

# Function to calculate audio features
def calculate_audio_features(audio, sr):
    # Calculate energy
    energy = np.array([sum(abs(audio[i:i + 1024] ** 2)) for i in range(0, len(audio), 1024)])
    
    # Calculate pitch
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sr)
    pitch = [pitches[magnitudes[:, i].argmax()] for i in range(magnitudes.shape[1])]
    
    # Calculate frequency (spectrum centroid)
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    
    # Calculate MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    
    # Calculate tempo
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    
    # Calculate zero-crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)[0]
    
    # Calculate chroma feature
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    
    return energy, pitch, spectral_centroids, mfccs, tempo, zero_crossing_rate, chroma

# Function to play audio
def play_audio(audio, sr, factor):
    slowed_audio = librosa.effects.time_stretch(audio, 1 / factor)
    sd.play(slowed_audio, sr // factor)
    sd.wait()  # Wait for the audio to finish playing
    sd.stop()

# Function to process and display graphs
def display_graphs(audio, sr, queue, factor):
    output_width = 1280  # Reduced width
    graph_height = 720  # Reduced height
    
    # Create an output video writer for audio features
    audio_video_output = 'output_audio_features.mp4'
    audio_fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    audio_fps = 30 // factor  # Reduced frames per second for the output video
    audio_out = cv2.VideoWriter(audio_video_output, audio_fourcc, audio_fps, (output_width, graph_height))

    # Number of frames to plot per second
    frames_per_second = int(sr / 1024 / factor)

    # Initialize lists to hold cumulative data for plotting
    cumulative_energy = []
    cumulative_pitch = []
    cumulative_spectral_centroids = []
    cumulative_mfccs = None
    cumulative_zero_crossing_rate = []
    cumulative_chroma = None

    # Process each frame for audio features
    frame_index = 0
    frame_duration = int(sr / frames_per_second)
    while frame_index * frame_duration < len(audio):
        current_frame_audio = audio[frame_index * frame_duration : (frame_index + 1) * frame_duration]
        energy, pitch, spectral_centroids, mfccs, tempo, zero_crossing_rate, chroma = calculate_audio_features(current_frame_audio, sr)
        
        # Append new data to cumulative lists
        cumulative_energy.extend(energy)
        cumulative_pitch.extend(pitch)
        cumulative_spectral_centroids.extend(spectral_centroids)
        cumulative_mfccs = np.hstack((cumulative_mfccs, mfccs)) if cumulative_mfccs is not None else mfccs
        cumulative_zero_crossing_rate.extend(zero_crossing_rate)
        cumulative_chroma = np.hstack((cumulative_chroma, chroma)) if cumulative_chroma is not None else chroma

        # Create a new figure for the audio features
        plt.figure(figsize=(16, 12))

        # Energy Graph
        plt.subplot(6, 1, 1)
        plt.plot(cumulative_energy, color='red')
        plt.ylabel('Energy')
        plt.xlim(0, len(cumulative_energy))

        # Pitch Graph
        plt.subplot(6, 1, 2)
        plt.plot(cumulative_pitch, color='blue')
        plt.ylabel('Pitch')
        plt.xlim(0, len(cumulative_pitch))

        # Spectral Centroid Graph
        plt.subplot(6, 1, 3)
        plt.plot(cumulative_spectral_centroids, color='green')
        plt.ylabel('Spectral Centroid')
        plt.xlim(0, len(cumulative_spectral_centroids))

        # MFCCs Graph
        plt.subplot(6, 1, 4)
        librosa.display.specshow(cumulative_mfccs, sr=sr, x_axis='time')
        plt.ylabel('MFCCs')

        # Zero-Crossing Rate Graph
        plt.subplot(6, 1, 5)
        plt.plot(cumulative_zero_crossing_rate, color='purple')
        plt.ylabel('Zero-Crossing Rate')
        plt.xlim(0, len(cumulative_zero_crossing_rate))

        # Chroma Feature Graph
        plt.subplot(6, 1, 6)
        librosa.display.specshow(cumulative_chroma, sr=sr, x_axis='time', y_axis='chroma')
        plt.ylabel('Chroma')

        plt.tight_layout()

        # Save the graph as an image
        plt.savefig('temp_audio_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
        plt.clf()

        graph = cv2.imread('temp_audio_graph.png')

        # Resize the graph to match the video width
        graph = cv2.resize(graph, (output_width, graph_height))

        # Write the graph to the output video
        audio_out.write(graph)

        # Display the graph in real-time
        cv2.imshow('Audio Graph', graph)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Increment the frame index
        frame_index += 1

    # Release the video writer for audio features
    audio_out.release()
    cv2.destroyAllWindows()
    queue.put('done')

# Function to download video from YouTube using yt-dlp
def download_video_from_youtube(url):
    video_file = 'video.mp4'
    audio_file = 'audio.wav'
    command_video = ['yt-dlp', '-f', 'bestvideo', '-o', video_file, url]
    command_audio = ['yt-dlp', '-f', 'bestaudio', '--extract-audio', '--audio-format', 'wav', '-o', 'audio.%(ext)s', url]
    
    subprocess.run(command_video, check=True)
    subprocess.run(command_audio, check=True)
    
    return video_file, audio_file

# Function to play YouTube video
def play_youtube_video(video_file, factor):
    cap = cv2.VideoCapture(video_file)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    delay = int(1000 / (fps / factor))
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('YouTube Video', frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    # YouTube URL
    youtube_url = "https://youtu.be/JFVVdTssEx0?si=yTV48kH3MwhWTZIx " 
    # url of high energy plots "https://youtu.be/Lp7E973zozc?si=4y_NHNxtXuZZjC72"
    # Replace with your desired YouTube URL
    
    # Download video and audio from YouTube
    video_path, audio_path = download_video_from_youtube(youtube_url)
    
    # Load the audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Slow down factor
    factor = 2

    # Create a queue to communicate between processes
    queue = Queue()

    # Create and start the audio playback process
    audio_process = Process(target=play_audio, args=(audio, sr, factor))
    audio_process.start()

    # Create and start the graph display process
    graph_process = Process(target=display_graphs, args=(audio, sr, queue, factor))
    graph_process.start()

    # Play YouTube video
    video_process = Process(target=play_youtube_video, args=(video_path, factor))
    video_process.start()

    # Wait for all processes to complete
    audio_process.join()
    graph_process.join()
    video_process.join()

    # Ensure the graph display process is done
    queue.get()
